In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

import sys, os
from pathlib import Path
# необходимо добавить путь до корня репозитория, чтобы работала библиотека fusionlib
sys.path.insert(0, os.path.join(Path('.').resolve().parent))

from fusionlib.predicts import predict # Функция, позволяет получить предсказание нейронки.
from fusionlib.check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита

# 1. Загрузка данных

In [2]:
# здесь и далее необходимо вставить пути, актуальные для Вашей машины
finetune_df_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/transactions_finetune.csv'
finetune_df = pd.read_csv(finetune_df_path)
print(f'Число уникальных пользователей для тюнинга модели: {finetune_df.user_id.nunique()}')

Число уникальных пользователей для тюнинга модели: 7080


In [3]:
finetune_targets_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/target_finetune.csv'
finetune_targets = pd.read_csv(finetune_targets_path)
pd.concat([
    finetune_targets.target.value_counts(normalize=True).rename("target_freq"), 
    finetune_targets.target.value_counts(normalize=False).rename("target_count")], axis=1)

,target_freq,target_count
0,0.962994,6818
1,0.037006,262


# 2. Получение валидационной выборки
В качестве примера получения валидационной выборки предлагаю взять 0.3 от набора транзакций для тюнинга модели.

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(test_size=.3, n_splits=1, random_state=7)
split = splitter.split(finetune_targets, finetune_targets.target)
_, val_inds = next(split)

val_targets = finetune_targets.iloc[val_inds]
val_user_id_set = set(val_targets.user_id.unique())
val_df = finetune_df[finetune_df.user_id.apply(lambda id: id in val_user_id_set)]

Удостоверимся, что распределение таргета осталось прежним

In [5]:
pd.concat([
    val_targets.target.value_counts(normalize=True).rename("target_freq"), 
    val_targets.target.value_counts(normalize=False).rename("target_count")], axis=1)

,target_freq,target_count
0,0.962806,2045
1,0.037194,79


In [6]:
val_df_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/validation_clear_transactions.csv'
val_df.to_csv(val_df_path, index_label=False)

# 3. Валидация решения
В зависимости от того, участвуете Вы в одном треке, или в обоих, предлагаю 2 решения:
- Валидация с базовым решением атаки. Этот метод наиболее близко коррелирует с лидербордом (на момент 1 половины соревнования). Позволяет начать работать над задачей `Защиты` с минимальными зананиями противоположного трека
- Валидация с кастомным решением атаки. Позволяет сравнить качество модели `Защиты` с собственными наработками по треку `Атака`. При минимальной доработке позволяет валидировать решение смежного трека

## 3.1. Валидация с базовым решением атаки

In [7]:
from fusionlib.validation import validate_model_
from fusionlib.predicts import reliable_predict

In [8]:
# путь до валидационной выборки
val_df_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/validation_clear_transactions.csv'
# путь до таргетов, которые используются на валидации
targets_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/target_finetune.csv'
# путь до файла с бинами после тренировки модели (nn_bins.pickle)
bins_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/models/nn_bins.pickle" 
# путь до файла с весами нейронной сети (nn_weights.ckpt)
model_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/models/nn_weights.ckpt" 
# путь до файла с квантилями для таргета (quantiles.pickle)
quantiles_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/misc/quantiles.json" 


In [9]:
%%time

mean_harm_roc_auc = validate_model_(reliable_predict, val_df_path, bins_path, model_path, targets_path, quantiles_path)
print(f'Качество на локальной валидации: {mean_harm_roc_auc}')

Global seed set to 20230206
100%|██████████| 2124/2124 [00:33<00:00, 63.20it/s]
Global seed set to 20230206


Качество на локальной валидации: 0.7159109481071629
CPU times: user 9min 50s, sys: 2min, total: 11min 50s
Wall time: 7min 49s


In [10]:
public_mean_harm_roc_auc = 0.705638	
print(f'Разница лидерборда и локальной валидации: {round(abs(public_mean_harm_roc_auc - mean_harm_roc_auc), 4)}')

Разница лидерборда и локальной валидации: 0.0103


## 3.2. Валидация с кастомным решением атаки
Предположим, что Вы атаковали отложенные на валидацию данные. Для оценки качества в задаче `Защита` сохраните атакованные данные и воспользуйтесь следующим методом: 

In [12]:
from fusionlib.validation import validate_model
fraud_val_df_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/validation_fraud_transactions.csv'
mean_harm_roc_auc = validate_model(reliable_predict, val_df_path, fraud_val_df_path, bins_path=bins_path, model_path=model_path, targets_path=targets_path)
print(f'Качество на локальной валидации: {mean_harm_roc_auc}')

Global seed set to 20230206
Global seed set to 20230206


Качество на локальной валидации: 0.7160533551043936
